In [201]:
import openpyxl
import pandas as pd


def read_excel(filename, nrows):
    book = openpyxl.load_workbook(filename=filename, read_only=True, data_only=True)
    first_sheet = book.worksheets[0]
    rows_generator = first_sheet.values

    header_row = next(rows_generator)
    data_rows = [row for (_, row) in zip(range(nrows - 1), rows_generator)]
    return pd.DataFrame(data_rows, columns=header_row)


churn_dataset = read_excel(r"C:\Users\irema\Desktop\turknetcalismaveriseti\turknetchurnekimanonim.xlsx", 1000)

In [202]:
# açıklama sütunundaki integer değeri çekme
churn_dataset["ACIKLAMA"]
i =0
integer = []

for row in churn_dataset['ACIKLAMA']:
    parcalar = row.split(" ")
    if parcalar[1] == "İletişim)" or parcalar[1] == "İletişim)(Yapa)":
        integer.append(parcalar[2])
    elif parcalar[1] == "Fiber)":
        integer.append(parcalar[2])
    else:
        integer.append(parcalar[1])
churn_dataset['ACIKLAMA']=integer
churn_dataset['ACIKLAMA'] =churn_dataset['ACIKLAMA'].astype(int)
churn_dataset['ACIKLAMA']

0      100
1      100
2       16
3       35
4      100
      ... 
994     16
995     35
996     16
997    100
998     75
Name: ACIKLAMA, Length: 999, dtype: int32

In [203]:
# tahmin edilecek sutündaki null değerleri çıkarma
churn_dataset = churn_dataset.dropna(subset = ['DURUM'])
churn_dataset

,NaN,ILTELKODU,ILADI,POSILCE,ILCE,CINSIYET,DURUM,ABONEBAS,CLOSE_DATE,KALDIGI_AY_SAYISI,...,DENVERGIRISTARIHI_3,DENVERCIKISTARIHI_3,DENVERKALDIGISURE_GUN_3,IKNATICKET_3,PORTERROR_SAYISI_3,MAX_SESSIONTIME_3,MIN_SESSIONTIME_3,TOTALUPLOADGB_3,TOTALDOWNLOADGB_3,CHURNTEST_DATE
0,0,212,İstanbul (Avrupa),939,SULTANGAZİ,E,A,2016-04-20,NaT,54,...,None,None,0,0,10,87108,0,16.5013,272.1967,2020-10-31
1,1,212,İstanbul (Avrupa),420,SARIYER,K,A,2016-04-20,NaT,54,...,None,None,0,0,1,90271,0,135.8656,185.5825,2020-10-31
2,2,262,Kocaeli,998,İZMİT,E,A,2016-04-21,NaT,54,...,None,None,0,0,3,90112,0,1.8255,46.6458,2020-10-31
3,3,212,İstanbul (Avrupa),418,KÜÇÜKÇEKMECE,E,A,2016-04-21,NaT,54,...,None,None,0,0,15,87588,1664,2.0288,42.0181,2020-10-31
4,4,212,İstanbul (Avrupa),425,BAYRAMPAŞA,E,A,2016-04-21,2020-10-02,54,...,None,None,0,0,0,0,0,0.0000,0.0000,2020-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,414,Şanlıurfa,1009,HALİLİYE,E,A,2016-05-09,NaT,53,...,None,None,0,0,5,86314,0,8.7980,155.0454,2020-10-31
995,995,216,İstanbul (Anadolu),422,ÜMRANİYE,E,A,2016-04-25,NaT,54,...,None,None,0,0,8,90237,0,23.3103,353.0814,2020-10-31
996,996,354,Yozgat,829,MERKEZ,E,A,2016-05-12,NaT,53,...,None,None,0,0,2,89372,5687,4.1911,104.4748,2020-10-31
997,997,212,İstanbul (Avrupa),412,EYÜP,E,A,2016-05-13,NaT,53,...,None,None,0,0,6,86570,0,8.2705,273.8619,2020-10-31


In [204]:
#'KAPASITE','CINSIYET','ODEMESEKLI', 'YAPA_VAE','FATURAGONDERIMTIPI' string sütunlarına encoding
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
#y = labelEncoder.fit_transform()

churn_dataset[['CINSIYET','ODEMESEKLI', 'YAPA_VAE','FATURAGONDERIMTIPI','DURUM']] =churn_dataset[['CINSIYET','ODEMESEKLI', 'YAPA_VAE','FATURAGONDERIMTIPI','DURUM']].apply(LabelEncoder().fit_transform)
churn_dataset[['CINSIYET','ODEMESEKLI', 'YAPA_VAE','FATURAGONDERIMTIPI','DURUM']]

,CINSIYET,ODEMESEKLI,YAPA_VAE,FATURAGONDERIMTIPI,DURUM
0,0,1,1,0,0
1,1,1,0,0,0
2,0,1,0,0,0
3,0,0,0,0,0
4,0,1,1,0,0
...,...,...,...,...,...
994,0,1,0,0,0
995,0,0,0,0,0
996,0,0,0,0,0
997,0,1,1,0,0


In [205]:
#drop duplicate columns
churn_dataset = churn_dataset.T.drop_duplicates().T
churn_dataset

,NaN,ILTELKODU,ILADI,POSILCE,ILCE,CINSIYET,DURUM,ABONEBAS,CLOSE_DATE,KALDIGI_AY_SAYISI,...,ADSLARIZA_TICKETKAPANMASURESI_3,ADSLARIZA_TICKETSL_3,INTERNETEBAGLANAMIYORUM_TICKETKAPANMASURESI_3,IKNATICKET_3,PORTERROR_SAYISI_3,MAX_SESSIONTIME_3,MIN_SESSIONTIME_3,TOTALUPLOADGB_3,TOTALDOWNLOADGB_3,CHURNTEST_DATE
0,0,212,İstanbul (Avrupa),939,SULTANGAZİ,0,0,2016-04-20,NaT,54,...,0,0,0,0,10,87108,0,16.5013,272.197,2020-10-31
1,1,212,İstanbul (Avrupa),420,SARIYER,1,0,2016-04-20,NaT,54,...,0,0,0,0,1,90271,0,135.866,185.583,2020-10-31
2,2,262,Kocaeli,998,İZMİT,0,0,2016-04-21,NaT,54,...,0,0,0,0,3,90112,0,1.8255,46.6458,2020-10-31
3,3,212,İstanbul (Avrupa),418,KÜÇÜKÇEKMECE,0,0,2016-04-21,NaT,54,...,0,0,0,0,15,87588,1664,2.0288,42.0181,2020-10-31
4,4,212,İstanbul (Avrupa),425,BAYRAMPAŞA,0,0,2016-04-21,2020-10-02,54,...,4668,0,0,0,0,0,0,0,0,2020-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,414,Şanlıurfa,1009,HALİLİYE,0,0,2016-05-09,NaT,53,...,0,0,0,0,5,86314,0,8.798,155.045,2020-10-31
995,995,216,İstanbul (Anadolu),422,ÜMRANİYE,0,0,2016-04-25,NaT,54,...,7982,1,0,0,8,90237,0,23.3103,353.081,2020-10-31
996,996,354,Yozgat,829,MERKEZ,0,0,2016-05-12,NaT,53,...,0,0,0,0,2,89372,5687,4.1911,104.475,2020-10-31
997,997,212,İstanbul (Avrupa),412,EYÜP,0,0,2016-05-13,NaT,53,...,0,0,0,0,6,86570,0,8.2705,273.862,2020-10-31


In [206]:
# NaN sütunu çıkar
churn_dataset = churn_dataset[set(list(churn_dataset)) - set([None])]
churn_dataset

,INTERNETEBAGLANAMIYORUM_TICKETKAPANMASURESI_1,OIMSIKAYET_DESTEKSAYISI_1,IKNATICKET_3,ADSLARIZA_TICKETSL_3,TOTALUPLOADGB_3,CM_TICKETKAPANMASURESI_2,ADSLARIZA_TICKETSL_2,CINSIYET,TOTALDOWNLOADGB_1,INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_1,...,CAGRIMERKEZIARAMASAYISI_2,POSILCE,HIZMETKANALI_SIKAYETSAYISI_1,ADSLARIZA_TICKETKAPANMASURESI_3,currentDown,TOTALDOWNLOADGB_2,ODEMESEKLI,TOTALUPLOADGB_2,FATURA_GECIKME_UCRETI_3,OIMSIKAYET_DESTEKSAYISI_2
0,0,0,0,0,16.5013,0,0,0,184.717,0,...,0,939,0,0,7774,227.03,1,11.5741,0,0
1,0,0,0,0,135.866,0,0,1,388.755,0,...,0,420,0,0,NaN,118.329,1,33.2407,0,0
2,0,0,0,0,1.8255,0,0,0,54.2985,0,...,0,998,0,0,16381,54.1644,1,2.3205,0,0
3,0,0,0,0,2.0288,0,0,0,64.473,0,...,0,418,0,0,35839,116.422,0,11.6085,0,0
4,0,2,0,0,0,0,1,0,0,0,...,0,425,0,4668,10535,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,0,8.798,0,0,0,242.352,0,...,0,1009,0,0,8120,231.596,1,7.8943,0,0
995,0,0,0,1,23.3103,0,0,0,356.038,0,...,0,422,0,7982,27331,164.473,0,9.9625,0,2
996,0,0,0,0,4.1911,0,0,0,97.0026,0,...,0,829,0,0,16383,63.7456,0,3.3803,0,0
997,0,0,0,0,8.2705,0,0,0,663.923,0,...,0,412,0,0,16977,795.86,1,33.3323,0,0


In [207]:
#boş ya da kullanışsız sütunları çıkar
churn_dataset=churn_dataset.drop(['MAX_SESSIONTIME_2','ABONEBAS','TOTALDOWNLOADGB_1','MIN_SESSIONTIME_2','CM_TICKETKAPANMASURESI_3','CM_TICKETKAPANMASURESI_1','INTERNETEBAGLANAMIYORUM_TICKETKAPANMASURESI_1','TOTALDOWNLOADGB_2','CM_TICKETKAPANMASURESI_2','MIN_SESSIONTIME_3','PORTERROR_SAYISI_1','ILTELKODU','currentDown','ADSLARIZA_TICKETKAPANMASURESI_2','ADSLARIZA_TICKETKAPANMASURESI_3','POSILCE','TDU_TICKETKAPANMASURESI_1','MAX_SESSIONTIME_1','TDU_TICKETKAPANMASURESI_3','MIN_SESSIONTIME_1','TOTALUPLOADGB_2','CLOSE_DATE','PORTERROR_SAYISI_2','PORTERROR_SAYISI_3','TDU_TICKETKAPANMASURESI_2','ILCE','INTERNETEBAGLANAMIYORUM_TICKETKAPANMASURESI_3','TOTALUPLOADGB_3','CHURNTEST_DATE','ADSLARIZA_TICKETKAPANMASURESI_1','TOTALUPLOADGB_1','MAX_SESSIONTIME_3','INTERNETEBAGLANAMIYORUM_TICKETKAPANMASURESI_2','ILADI','TOTALDOWNLOADGB_3'],axis =1)
churn_dataset = churn_dataset[set(list(churn_dataset))-set(['DENVERGIRISTARIHI_3'])-set(['DENVERGIRISTARIHI_1'])]
churn_dataset


,IKNATICKET_3,OIMSIKAYET_DESTEKSAYISI_1,ADSLARIZA_TICKETSL_3,FATURA_GECIKME_UCRETI_1,ADSLARIZA_TICKETSL_2,CINSIYET,FATURA_GECIKME_UCRETI_2,INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_1,CM_DESTEKSAYISI_3,DURUM,...,IKNATICKET_2,OIMSIKAYET_DESTEKSAYISI_3,RISKLIMUSTERI,IKNATICKET_1,CAGRIMERKEZIARAMASAYISI_2,ADSLARIZA_DESTEKSAYISI_1,HIZMETKANALI_SIKAYETSAYISI_1,ODEMESEKLI,FATURA_GECIKME_UCRETI_3,OIMSIKAYET_DESTEKSAYISI_2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,0,0,1,0,0,0,0,0,...,0,2,0,2,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [208]:
list(churn_dataset) #49 sütun kaldı

['IKNATICKET_3',
 'OIMSIKAYET_DESTEKSAYISI_1',
 'ADSLARIZA_TICKETSL_3',
 'FATURA_GECIKME_UCRETI_1',
 'ADSLARIZA_TICKETSL_2',
 'CINSIYET',
 'FATURA_GECIKME_UCRETI_2',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_1',
 'CM_DESTEKSAYISI_3',
 'DURUM',
 'ADSLARIZA_DESTEKSAYISI_3',
 'ARKADASINIGETIR',
 'DESTEKSAYISI_2',
 'CM_DESTEKSAYISI_2',
 'FATURA_GECIKME_3',
 'FATURA_GECIKME_2',
 'ADSLARIZA_TICKETSL_1',
 'DESTEKSAYISI_1',
 'KALDIGI_AY_SAYISI',
 'FATURA_GECIKME_1',
 'TDU_DESTEKSAYISI_3',
 'YAPA_VAE',
 'KAPASITE',
 'ADSLARIZA_DESTEKSAYISI_2',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_3',
 'CAGRIMERKEZIARAMASAYISI_1',
 'TDU_DESTEKSAYISI_1',
 'ARKADASINIGETIRILEGELENLER',
 'TDU_TICKETSL_1',
 'DESTEKSAYISI_3',
 'TDU_TICKETSL_3',
 'TDU_TICKETSL_2',
 'TDU_DESTEKSAYISI_2',
 'FATURAGONDERIMTIPI',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_2',
 'CM_TICKETSL_3',
 'ACIKLAMA',
 'CM_DESTEKSAYISI_1',
 'CAGRIMERKEZIARAMASAYISI_3',
 'IKNATICKET_2',
 'OIMSIKAYET_DESTEKSAYISI_3',
 'RISKLIMUSTERI',
 'IKNATICKET_1',
 'CAG

In [146]:
train_columns = list(set(list(churn_dataset))-set(['DURUM']))  #durum dışındaki bütün sütunlar
looked_column = list(set(['DURUM']))                           #tahmin edilecek

y = churn_dataset[looked_column]
train_columns

['OIMSIKAYET_DESTEKSAYISI_1',
 'IKNATICKET_3',
 'ADSLARIZA_TICKETSL_3',
 'FATURA_GECIKME_UCRETI_1',
 'ADSLARIZA_TICKETSL_2',
 'CINSIYET',
 'FATURA_GECIKME_UCRETI_2',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_1',
 'CM_DESTEKSAYISI_3',
 'ADSLARIZA_DESTEKSAYISI_3',
 'DESTEKSAYISI_2',
 'CM_DESTEKSAYISI_2',
 'ADSLARIZA_TICKETSL_1',
 'FATURA_GECIKME_3',
 'FATURA_GECIKME_2',
 'ARKADASINIGETIR',
 'DESTEKSAYISI_1',
 'KALDIGI_AY_SAYISI',
 'FATURA_GECIKME_1',
 'TDU_DESTEKSAYISI_3',
 'YAPA_VAE',
 'KAPASITE',
 'ADSLARIZA_DESTEKSAYISI_2',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_3',
 'CAGRIMERKEZIARAMASAYISI_1',
 'TDU_DESTEKSAYISI_1',
 'ARKADASINIGETIRILEGELENLER',
 'TDU_TICKETSL_1',
 'DESTEKSAYISI_3',
 'TDU_TICKETSL_3',
 'TDU_TICKETSL_2',
 'TDU_DESTEKSAYISI_2',
 'FATURAGONDERIMTIPI',
 'INTERNETEBAGLANAMIYORUM_DESTEKSAYISI_2',
 'CM_TICKETSL_3',
 'ACIKLAMA',
 'CM_DESTEKSAYISI_1',
 'CAGRIMERKEZIARAMASAYISI_3',
 'IKNATICKET_2',
 'OIMSIKAYET_DESTEKSAYISI_3',
 'RISKLIMUSTERI',
 'IKNATICKET_1',
 'CAGRIMERKEZIA

In [209]:
from collections import Counter
Counter(churn_dataset['DURUM'])
Counter(churn_dataset['ARKADASINIGETIR'])


Counter({nan: 1,
         nan: 1,
         nan: 1,
         3.0: 55,
         4.0: 49,
         1.0: 208,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         7.0: 8,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         2.0: 124,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         8.0: 10,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
       

In [210]:
churn_dataset['ARKADASINIGETIR'] =churn_dataset['ARKADASINIGETIR'].fillna(0) #sadece bu sütunda NaN değer var
churn_dataset['ARKADASINIGETIR'] =churn_dataset['ARKADASINIGETIR'].astype(int)
Counter(churn_dataset['ARKADASINIGETIR'])

Counter({0: 500,
         3: 55,
         4: 49,
         1: 208,
         7: 8,
         2: 124,
         8: 10,
         6: 18,
         5: 12,
         20: 2,
         29: 1,
         13: 1,
         9: 2,
         10: 2,
         28: 1,
         67: 1,
         60: 1,
         16: 2,
         41: 1,
         11: 1})

In [199]:
import statsmodels.api as sm
import numpy as np

X = churn_dataset[train_columns]
y = churn_dataset[looked_column]

cols = list(train_columns)
pmax = 1
while (len(cols)>0):
    p=[]
    X_1 = churn_dataset[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y.astype(float),X_1.astype(float)).fit()
    p = pd.Series(model.pvalues.values[1:].index ==cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
        
selected_features = cols
len(selected_features)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).